In [ ]:
!pip install waymo-open-dataset-tf-2-11-0==1.6.1
!pip install timm

import math
import os
import uuid
import time
import glob
import sys


import pandas as pd
import numpy as np
import itertools
import tensorflow as tf
import cv2

import matplotlib as plt


from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

In [ ]:
!git clone https://github.com/GummaeN/WaymoOpenMotionPred2023

In [3]:
import sys
sys.path.insert(0,'/content/WaymoOpenMotionPred2023/src')
from tf_features import roadgraph_features, get_tffeatures
import rasterize

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#feature description for tf_records
features_description = get_tffeatures()

In [6]:
def load_tf_files(dir, features_description):
  files = glob.glob(dir, recursive=False)
  print(f"Number of tf_example files: {len(files)}")


  dataset = tf.data.TFRecordDataset(files, compression_type='')
  data = next(dataset.as_numpy_iterator())
  parsed = tf.io.parse_single_example(data, features_description)

  print(f"Len of scene: {len(parsed)}")
  print(f"Keys: {parsed.keys()}")


  print(f"\nCurrent frames: {len(parsed['state/current/x'][0])}")
  print(f"Past frames: {len(parsed['state/past/x'][0])}")
  print(f"Future frames: {len(parsed['state/future/x'][0])}")
  return dataset

In [7]:
dir = '/content/drive/MyDrive/WaymoOpenDataset/Motion2023/train/'
file_pattern = 'uncompressed_tf_example_training_training_tfexample.tfrecord-00*'
dataset = load_tf_files(dir+file_pattern,features_description)

Number of tf_example files: 50
Len of scene: 63
Keys: dict_keys(['roadgraph_samples/dir', 'roadgraph_samples/id', 'roadgraph_samples/type', 'roadgraph_samples/valid', 'roadgraph_samples/xyz', 'scenario/id', 'state/current/bbox_yaw', 'state/current/height', 'state/current/length', 'state/current/timestamp_micros', 'state/current/valid', 'state/current/vel_yaw', 'state/current/velocity_x', 'state/current/velocity_y', 'state/current/width', 'state/current/x', 'state/current/y', 'state/current/z', 'state/future/bbox_yaw', 'state/future/height', 'state/future/length', 'state/future/timestamp_micros', 'state/future/valid', 'state/future/vel_yaw', 'state/future/velocity_x', 'state/future/velocity_y', 'state/future/width', 'state/future/x', 'state/future/y', 'state/future/z', 'state/id', 'state/is_sdc', 'state/past/bbox_yaw', 'state/past/height', 'state/past/length', 'state/past/timestamp_micros', 'state/past/valid', 'state/past/vel_yaw', 'state/past/velocity_x', 'state/past/velocity_y', 'stat

In [ ]:

data_it = dataset.as_numpy_iterator()
os.makedirs('/content/traintar')

for i, data in enumerate(data_it):

  if i % 200 == 9:
    print(f"Scenario:{i}")

  parsed = tf.io.parse_single_example(data, features_description)
  params = rasterize.load_params(parsed)

  for i in range(len(params['tracks_to_pred'])):
    if params['tracks_to_pred'][i] > 0:
      track_to_pred = np.zeros(128)
      track_to_pred[i] = 1
      raster = rasterize.rasterize(params,track_to_pred)
      rasterize.save_raster(raster,i)

In [ ]:
!tar -zcvf 'train.tar.gz' -C / content/traintar
from google.colab import files
files.download('/content/train.tar.gz')
